In [1]:
%matplotlib qt5
# %matplotlib inline

In [391]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
from scipy.optimize import curve_fit


from qick import *
from qick.helpers import gauss

import time
import os
import sys
sys.path.append('/home/xilinx/jupyter_notebooks/')
import scipy as sp
import json
from scipy.fft import fft, fftfreq
import Pyro4.util

from slab.instruments import *
from slab.experiment import Experiment
from slab.datamanagement import SlabFile
from slab import get_next_filename, AttrDict

# Figure params
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [10,6]
plt.rcParams.update({'font.size': 14})
mpl.rcParams['figure.dpi'] = 150

default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
linestyle_cycle=['solid', 'dashed', 'dotted', 'dashdot']
marker_cycle = ['o', '*', 's', '^']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load experiments

In [3]:
# %aimport experiments
# NOTE: ADDING NEW METHODS WILL NOT BE UPDATED
import experiments as meas

imported experiments.clifford_averager_program
imported experiments.fitting
imported experiments.four_qubit.fourQ_state_tomo
imported experiments.qram_protocol_timestepped
imported experiments.single_qubit.acstarkshift_spectroscopy
imported experiments.single_qubit.amplitude_rabi
imported experiments.single_qubit.length_rabi
imported experiments.single_qubit.pulse_probe_spectroscopy
imported experiments.single_qubit.resonator_spectroscopy
imported experiments.single_qubit.rfsoc_tof_calibration
imported experiments.single_qubit.single_shot
imported experiments.single_qubit.t1
imported experiments.single_qubit.t2_ramsey
imported experiments.three_qubit.threeQ_state_tomo
imported experiments.two_qubit.amplitude_rabi_EgGf
imported experiments.two_qubit.amplitude_rabi_f0g1
imported experiments.two_qubit.amprabi_opt_ctrl_state_prep
imported experiments.two_qubit.crosstalk_echo_calib
imported experiments.two_qubit.length_rabi_EgGf
imported experiments.two_qubit.length_rabi_F0G1
imported exper

### Set save data

In [4]:
save_data = False

In [5]:
def prev_data(expt_path, filename=None):
    """Reopen saved data"""
    temp_data_file = expt_path
    if filename is not None: temp_data_file = os.path.join(expt_path, filename)
    print(temp_data_file)
    with SlabFile(temp_data_file) as a:
        attrs = dict()
        for key in list(a.attrs):
            attrs.update({key:json.loads(a.attrs[key])})
        keys = list(a)
        temp_data = dict()
        for key in keys:
            temp_data.update({key:np.array(a[key])})
        print(f'Extracted data and attrs {list(a.attrs)}')
    return temp_data, attrs

In [6]:
data_path = 'S:\\QRAM\\qram_4QR2\\'
expt_path = os.path.join(data_path, 'data', 'data_241025')
print('Data will be stored in', expt_path)

Data will be stored in S:\QRAM\qram_4QR2\data\data_241025


In [7]:
config_file = 'config_q3diamond_full688and638_reset.yml'
config_path = os.path.join('s:\\Connie\\experiments\\qram_tprocv1_expts\\configs\\', config_file)
print('Config will be', config_path)

Config will be s:\Connie\experiments\qram_tprocv1_expts\configs\config_q3diamond_full688and638_reset.yml


# Load config

In [320]:
import yaml
with open(config_path, 'r') as cfg_file:
    yaml_cfg = yaml.safe_load(cfg_file)
yaml_cfg = AttrDict(yaml_cfg)

# Setup experiment hardware

Set in config:
- LO alias, frequency, power
- SOC dac channels, nyquist zones

In [9]:
im = InstrumentManager(ns_address='10.108.30.75') # Spilker Fermium
print(list(im))

['Pyro.NameServer', 'Qick_QRAM']


In [10]:
soc = QickConfig(im[yaml_cfg['aliases']['soc']].get_cfg())
print(soc)

QICK running on ZCU216, software version 0.2.272

Firmware configuration (built Sat Oct  5 10:21:47 2024):

	Global clocks (MHz): tProcessor 430.080, RF reference 245.760

	7 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 0 is 0_230, on JHC3
	1:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 1 is 1_230, on JHC4
	2:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 2 is 2_230, on JHC3
	3:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 3 is 3_230, on JHC4
	4:	axis_signal_gen_v6 - envelope memory 32768 samples (4.762 us)
		fs=6881.280 MHz, fabric=430.080 MH

## Parameter

In [11]:
qubit_i = 2

# Rabi versus amplitude 

We first want to know the relationship between the gain and rabi drive strength

In [225]:
# gain_start = 100 
# gain_stop = 1000
# gain_pts = 5
# gain_vec = np.linspace(gain_start, gain_stop, gain_pts)

reps = 1
rounds = 100 

t_start = 0.1
t_stop = 0.5 # us
t_pts = 300
t_step = (t_stop - t_start) / t_pts
print(f'Time step is {t_step} us')
t_step = soc.cycles2us(soc.us2cycles(t_step))
print(f'Time step is {t_step} cycles')

rabi_freq_vec = np.zeros(gain_pts)
rabi_freq_vec_err = np.zeros(gain_pts)

save_data = True

Time step is 0.0013333333333333335 us
Time step is 0.0023251488095238095 cycles


NameError: name 'gain_pts' is not defined

In [105]:

RabiFreqExpt = Experiment(
                        path=expt_path,
                        prefix=f"rabi_freq_{qubit_i}",
                        config_file=config_path)

RabiFreqExpt.data = dict(amps=[], times=[], rabi_freq = [], gain_pts=[])
RabiFreqExpt.data['gain_pts'] = gain_vec

time_vec = []
amps_vec = []


for idx_g, g in enumerate(gain_vec):
    print(f"Gain {g} ({idx_g+1}/{len(gain_vec)})")

    lengthrabi = meas.LengthRabiExperiment(
        soccfg=soc,
        path=expt_path,
        prefix=f"length_rabi_qubit{qubit_i}",
        config_file=config_path,
    )
    lengthrabi.cfg = AttrDict(deepcopy(yaml_cfg))
    
    lengthrabi.cfg.device.qubit.pulses.pi_ge.half_gain[qubit_i*5] = int(g)

    lengthrabi.cfg.expt = dict(
        start=t_start, # pulse length (sigma) [us]
        step=t_step, # [us]
        expts=t_pts,
        reps=reps*rounds,
        pulse_type='gauss',
        checkEF=False, 
        qTest=qubit_i,
        qZZ=None,
    )

    import Pyro4.util
    try:
        lengthrabi.go(analyze=False, display=False, progress=True, save=False)
        
    except Exception:
        print("Pyro traceback:")
        print("".join(Pyro4.util.getPyroTraceback()))
        

    amps = lengthrabi.data['amps']
    time = lengthrabi.data['xpts']
    # compute FFT
    amp_fft = fft(amps)
    freq = fftfreq(len(amps), time[1] - time[0])
    
    # find the peak
    idx = np.argmax(np.abs(amp_fft[1:])) + 1
    freq_peak = freq[idx]
    

    rabi_freq_vec[idx_g] = freq_peak/2

    
    if save_data:
        time_vec.append(time)
        amps_vec.append(amps)
        
        
    


NameError: name 'gain_vec' is not defined

In [ ]:
# fit with a linear function

def linear(x, a, b):
    return a*x + b

popt, pcov = curve_fit(linear, gain_vec, rabi_freq_vec)

a = popt[0]
a_err = np.sqrt(pcov[0,0])
b = popt[1]
b_err = np.sqrt(pcov[1,1])


if save_data:
    RabiFreqExpt.data['times'] = time_vec
    RabiFreqExpt.data['amps'] = amps_vec
    # divide by 2 to get the Rabi frequency ( given we apply two pi/2 actually)
    RabiFreqExpt.data['rabi_freq'] = rabi_freq_vec
    RabiFreqExpt.data['fit'] = [a, b]
    RabiFreqExpt.data['fit_err'] = [a_err, b_err]
    RabiFreqExpt.save_data()

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
ax.plot(gain_vec, rabi_freq_vec, 'o-')
ax.plot(gain_vec, linear(gain_vec, *popt), '--', color='black')

text = f"Fit: {a*1000:.2f} x/1000 + {b:.2f}\n"
ax.text(0.1, 0.9, text, transform=ax.transAxes, fontsize=12, verticalalignment='top')

ax.set_xlabel('Gain')
ax.set_ylabel('Rabi frequency (MHz)')
ax.set_title(f'Rabi frequency vs Gain for qubit {qubit_i}')
fig.tight_layout()

In [196]:
filename = yaml_cfg.device.qubit.pulses.pihalf_ge_robust.filename[qubit_i]
path = os.path.join('S:\QRAM\qram_4QR2\optctrl_pulses', f"{filename}.npz")
nb_qb = 4


# open the file
scaling = 1

with np.load(path) as data:
    times = data['times']*scaling
    I = []
    Q = []
    for i in range(nb_qb):
        if 'I_'+str(i) in data:
            I.append(data['I_'+str(i)]/1e6/scaling)
            Q.append(data['Q_'+str(i)]/1e6/scaling)
        else:
            I.append(np.zeros(times.shape))
            Q.append(np.zeros(times.shape))

In [197]:
fig, ax = plt.subplots(figsize=(4,3))

for i in range(len(I)):
    ax.plot(times, I[i], label=f'I{i}')
    ax.plot(times, Q[i], label=f'Q{i}')
    
ax.set_xlabel('Time (us)')
ax.set_ylabel('Amplitude (MHz)')

ax.legend()
fig.tight_layout()
    

In [ ]:
# for all qubit take the maximum of the I and Q

gain = np.zeros(len(I), dtype=int)

for i in range(len(I)):
    amp_I = np.max(np.abs(I[i]))
    amp_Q = np.max(np.abs(Q[i]))
    amp_max = np.max([amp_I, amp_Q])
    if amp_max == 0:
        gain[i] = 0
    else:
        gain_max = (amp_max - b) / a
        # for some reason the gain we need to set is twice the value
        gain[i] = gain_max*2
        
        

# First round of error amplification to get the gain right

In [198]:
gain_scale = np.linspace(0.1, 1.5, 15)

x_tab = []
p_tab = []

qubit_i=0
gain = 5000

with open(config_path, 'r') as cfg_file:
    yaml_cfg = yaml.safe_load(cfg_file)
yaml_cfg = AttrDict(yaml_cfg)


for s in gain_scale:

    npulsecalib = meas.NPulseExperiment(
        soccfg=soc,
        path=expt_path,
        prefix=f"NPulseExptQ{qubit_i}",
        config_file=config_path,
    )

    npulsecalib.cfg = AttrDict(deepcopy(yaml_cfg))
    npulsecalib.cfg.device.qubit.pulses.pihalf_ge_robust.gain[qubit_i*5] = int(gain*s)
    # npulsecalib.cfg.device.qubit.pulses.pihalf_ge_robust.gain[qubit_i*5] = int(1)
    # npulsecalib.cfg.device.qubit.f_ge_robust[qubit_i*5] = 3448.5530242338505

    npulsecalib.cfg.expt = dict(
        start=1, # number gate repetitions
        step=10,
        expts=1,
        reps=1000,
        loops=10,
        pulse_type='robust',
        use_robust_pulses=False, 
        checkEF=False, 
        qTest=qubit_i,
        qZZ=None,
        test_pi_half=True,
        error_amp=True,
        skip_first_pi2=True,
        post_process='threshold',
        singleshot_reps=5000,
        resonator_reset=[0, 2, 3],
        full_mux_expt=True
    )

    # print(npulsecalib.cfg)
    import Pyro4.util
    try:
        npulsecalib.acquire(progress=True, debug=False)
    except Exception:
        print("Pyro traceback:")
        print("".join(Pyro4.util.getPyroTraceback()))
        
        
    data = npulsecalib.data
    
    p_vec = data['avgi'][-1]
    x_vec = data['xpts'][-1]
    
    p_tab.append(p_vec)
    x_tab.append(x_vec)

  0%|          | 0/2 [00:00<?, ?it/s]

Qubit  (0)
g unrotated averages:
Amps 134.68671037460825 +/- 63.859747430686134
e unrotated averages:
Amps 293.9274240477171 +/- 71.55493237982458
ge fidelity (%): 79.12 	 angle (deg): 0.0 	 threshold ge: 213.10212188356778


  0%|          | 0/10 [00:00<?, ?it/s]

CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay


  0%|          | 0/2 [00:00<?, ?it/s]

...

CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay


  0%|          | 0/2 [00:00<?, ?it/s]

Qubit  (0)
g unrotated averages:
Amps 139.19774680080894 +/- 67.35846395060236
e unrotated averages:
Amps 280.33102573284117 +/- 75.00815982142753
ge fidelity (%): 74.25759551910387 	 angle (deg): 0.0 	 threshold ge: 203.40227233977197


  0%|          | 0/10 [00:00<?, ?it/s]

CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay


In [199]:
# fig, ax = plt.subplots(figsize=(6,4))   

# for i in range(len(p_tab)):
#     ax.plot(x_tab[i], p_tab[i], label=f'Gain {gain_scale[i]:.2f}')

# ax.set_xlabel('Number of pulses')
# ax.set_ylabel('Average intensity')
# ax.legend()

fig, ax = plt.subplots(figsize=(6,4))

ax.plot(gain_scale, p_tab, 'o-')
    

In [201]:
gain_corr =  gain*1.5
# save the new gain in the yaml file
yaml_cfg.device.qubit.pulses.pihalf_ge_robust.gain[qubit_i*5] = int(gain_corr)

In [202]:
gain_corr

7500.0

# $\pi$-$\pi$ Exp

In [203]:
qubit_i= 0


In [204]:
piminuspi.cfg.device.qubit.f_ge[qubit_i*4 + qubit_i]-span_f/2

4106.097866562327

In [335]:
piminuspi = meas.PiMinusPiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"PiMinusPiExpt{qubit_i}",
    config_file=config_path,
)
reps = 100
rounds = 1
span_f = 10
npts_f = 20

# with open(config_path, 'r') as cfg_file:
#     yaml_cfg = yaml.safe_load(cfg_file)
# yaml_cfg = AttrDict(yaml_cfg)
piminuspi.cfg = AttrDict(deepcopy(yaml_cfg))
print(piminuspi.cfg.device.qubit.f_ge_robust[qubit_i*4 + qubit_i])



piminuspi.cfg.expt = dict(
    start_N=0, # number gate repetitions
    step_N=1,
    expts_N=10,
    start_f=piminuspi.cfg.device.qubit.f_ge[qubit_i*4 + qubit_i]-span_f/2,
    step_f=span_f/npts_f,
    expts_f=npts_f+1,
    reps=int(reps*rounds),
    loops=1,
    pulse_type='robust',
    use_robust_pulses=True, 
    test_pi_half=True, 
    checkEF=False,
    qTest=qubit_i,
    qZZ=None,
    post_process='threshold',
    singleshot_reps=15000,
    full_mux_expt=True,
    resonator_reset = [0, 2, 3]
)

# print(piminuspi.cfg)
import Pyro4.util
try:
    piminuspi.acquire(progress=True, debug=True)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

4107.508010580633


  0%|          | 0/2 [00:00<?, ?it/s]

Qubit  (0)
ge fidelity (%): 74.62571371424762 	 angle (deg): 0.0 	 threshold ge: 204.19460594464456
thresholds=[204.19460594464456, 0, 0, 0],
angles=[0.0, 0, 0, 0],
ge_avgs=[[-0.6256753458096013, -0.6516774613506915, -1.209445728234337, -0.4825819907784108], array([0., 0., 0., 0.]), array([0., 0., 0., 0.]), array([0., 0., 0., 0.])],
counts_calib=[[13277.0, 1723.0], [2108.0, 12892.0]]


  0%|          | 0/10 [00:00<?, ?it/s]

CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am 

In [336]:
data = piminuspi.data
piminuspi.analyze(fit=True)
piminuspi.display(fit=True)

yaml_cfg.device.qubit.f_ge_robust = np.reshape(yaml_cfg.device.qubit.f_ge_robust, (4,4))
yaml_cfg.device.qubit.f_ge_robust[qubit_i, qubit_i] = data["best_freq"]
# results_Qi.update({'Qubit freq':yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
# print(f'*Set qubit {qubit_i} freq to {yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i]}*')
# print(results_Qi)
print(f'*New f_ge\n', yaml_cfg.device.qubit.f_ge_robust)
yaml_cfg.device.qubit.f_ge_robust = yaml_cfg.device.qubit.f_ge_robust.flatten().tolist()
print(f'*New f_ge flattened\n{yaml_cfg.device.qubit.f_ge_robust}')



Attempted to init fitparam 3 to 0.0, which is out of bounds 0 to 0.34091054545144245. Instead init to 0.17045527272572122
Fit best freq 4111.788818855827 which is 4.280808275194431 away from old freq 4107.508010580633
*New f_ge
 [[4111.78881886 4108.14770152 4114.31037246 4114.31037246]
 [3443.00323034 3448.34584963    0.            0.        ]
 [4763.941      4763.941      4763.941      4763.941     ]
 [4384.25034226 4384.25034226 4384.25034226 4384.25034226]]
*New f_ge flattened
[4111.788818855827, 4108.147701523904, 4114.310372455052, 4114.310372455052, 3443.0032303441935, 3448.345849629168, 0.0, 0.0, 4763.941, 4763.941, 4763.941, 4763.941, 4384.25034225973, 4384.25034225973, 4384.25034225973, 4384.25034225973]


In [ ]:
piminuspi.save_data()

# Second round of error amplification

In [50]:
qubit_i = 1
# with open(config_path, 'r') as cfg_file:
#     yaml_cfg = yaml.safe_load(cfg_file)
# yaml_cfg = AttrDict(yaml_cfg)
# gain = yaml_cfg.device.qubit.pulses.pihalf_ge_robust.gain[qubit_i*5]
# print(yaml_cfg.device.qubit.f_ge_robust[qubit_i*5])
# print(f'New gain for qubit {qubit_i} is {gain}')

In [55]:
npulsecalib = meas.NPulseExperiment(
        soccfg=soc,
        path=expt_path,
        prefix=f"NPulseExptQ{qubit_i}",
        config_file=config_path,
    )


# # 



npulsecalib.cfg = AttrDict(deepcopy(yaml_cfg))
npulsecalib.cfg.device.qubit.pulses.pihalf_ge_robust.gain[1] = int(7057.739272231432)
 

npulsecalib.cfg.expt = dict(
    start=1, # number gate repetitions
    step=1,
    expts=20,
    reps=5000,
    loops=1,
    pulse_type='robust',
    checkEF=False, 
    qTest=qubit_i,
    qZZ=None,
    test_pi_half=True,
    error_amp=True,
    skip_first_pi2 = False, 
    post_process='threshold',
    singleshot_reps=10000,
    resonator_reset=[0, 2, 3], 
    full_mux_expt = True,
    use_robust_pulses=True
)

# print(npulsecalib.cfg)
import Pyro4.util
try:
    npulsecalib.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

  0%|          | 0/2 [00:00<?, ?it/s]

Qubit  (1)
g unrotated averages:
Amps 1355.037338751546 +/- 314.4256074745757
e unrotated averages:
Amps 1860.0010723223527 +/- 331.7884643386607
ge fidelity (%): 58.048082808280846 	 angle (deg): 0.0 	 threshold ge: 1605.6006088527329


  0%|          | 0/20 [00:00<?, ?it/s]

CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am adding a delay
CAREFUll I am 

In [56]:
data = npulsecalib.data
npulsecalib.analyze(fit=True, scale=None)
npulsecalib.display(fit=True, scale=None)

fitparams [None, 0.017453292519943295]
Attempted to init fitparam 0 to 0.0, which is out of bounds 0.0 to 0.0. Instead init to 0.0
From amps: adjust amplitude to 4456 / 1.0018072645084664 = 4447.961357303915
From avgi: adjust amplitude to 4456 / 1.0018072645084664 = 4447.961357303915
From avgq: adjust amplitude to 4456 / 1.000193925472444 = 4455.136035639487


# Test with 1Q tomo

In [57]:
sys.path.append(os.getcwd()+'/../../qutip_sims')
from QSwitch import QSwitch
from PulseSequence import PulseSequence
from TomoAnalysis import TomoAnalysis
tomo_analysis = TomoAnalysis(nb_qubits=1)
import qutip as qt

meas_order = np.array(['Z', 'X', 'Y'])
psiZ = [qt.basis(2,0), qt.basis(2,1)]
psiX = [1/np.sqrt(2)*(psiZ[0]+psiZ[1]), 1/np.sqrt(2)*(psiZ[0]-psiZ[1])]
psiY = [1/np.sqrt(2)*(psiZ[0]+1j*psiZ[1]), 1/np.sqrt(2)*(psiZ[0]-1j*psiZ[1])]
psi_dict = dict(Z=psiZ, X=psiX, Y=psiY)

In [144]:
tomo_analysis = TomoAnalysis(nb_qubits=1)
meas_order = np.array(['Z', 'X', 'Y'])
psiZ = [qt.basis(2,0), qt.basis(2,1)]
psiX = [1/np.sqrt(2)*(psiZ[0]+psiZ[1]), 1/np.sqrt(2)*(psiZ[0]-psiZ[1])]
psiY = [1/np.sqrt(2)*(psiZ[0]+1j*psiZ[1]), 1/np.sqrt(2)*(psiZ[0]-1j*psiZ[1])]
psi_dict = dict(Z=psiZ, X=psiX, Y=psiY)



qubit = 1 # which qubit/resonators to do tomo on
init_state = '|0>|0+1>' # switch, input
play_pulses = [0]

pi_half_swaps = None
cool_qubits = None
add_phase = False
readout_cool = False
use_IQ_pulse = False
use_robust_pulses = True # use robust pulses for state prep + tomo
ZZ_qubit = None

print('play_pulses', play_pulses)

apply_ps = False
ps_adjust = [np.inf]*4
ps_qubits = [0, 1, 2, 3]
psiZ = [qt.basis(2,0), qt.basis(2,1)]
psi_id = (psiZ[1] + psiZ[0]).unit()
rho_id = qt.ket2dm(psi_id)
print(rho_id)

play_pulses [0]
Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.5 0.5]
 [0.5 0.5]]


In [145]:
# saved_files = []
qramExpt = meas.QramProtocol1QTomoExperiment(
    soccfg=soc,
    path=expt_path,
    config_file=config_path,
)

span = np.inf
npts = 1
if span < np.inf: print('time step size [cycles]:', soc.us2cycles(span/npts))

qramExpt.cfg.expt = dict(
    start=span,
    step=0,
    expts=1, # number of different time experiments
    reps=40000, # number of reps per tomo/time step,
    # reps=2000000, # number of reps per tomo/time step,
    singleshot_reps=10000, # reps per state for singleshot calibration
    post_process='threshold', 
    qubit=qubit,
    init_state=init_state,
    play_pulses=play_pulses,
    cool_qubits=cool_qubits,
    use_IQ_pulse=use_IQ_pulse, # state transfer IQ pulses for the state prep
    use_robust_pulses=use_robust_pulses, # use robust pulses for state prep + tomo
    plot_IQ=False,
    add_phase=add_phase,
    ZZ_qubit=ZZ_qubit,
    pi_half_swaps=pi_half_swaps,
    resonator_reset=[0, 2, 3],
    full_mux_expt = True,  

    
)

try:
    qramExpt.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

  0%|          | 0/2 [00:00<?, ?it/s]

Qubit (1) ge
ge fidelity (%): 54.88777377737774 	 angle (deg): 0.0 	 threshold ge: 1581.9235131079479
thresholds=[0, 1581.9235131079479, 0, 0],
angles=[0, 0.0, 0, 0],
ge_avgs=[[0, 0, 0, 0], [1096.7724204231083, 757.0969433685924, 1525.131147030106, 969.9683392188773], [0, 0, 0, 0], [0, 0, 0, 0]] ,
counts_calib=[[7619.0, 2381.0], [2173.0, 7827.0]]


  0%|          | 0/3 [00:00<?, ?it/s]

In [146]:
tomo = qramExpt


In [147]:
data = tomo.data
cfg = tomo.cfg
init_state = tomo.cfg.expt.init_state
qubit = tomo.cfg.expt.qubit
pulse_dict = tomo.pulse_dict
assert np.all(np.array(tomo.meas_order) == tomo_analysis.meas_order)

print('meas_order', meas_order)

n_tomo_raw, n_calib = tomo_analysis.n_tomo_calib_from_data(data=data, cfg=cfg, ps_adjust=ps_adjust, ps_qubits=ps_qubits, tomo_qubits=[qubit], apply_ps=apply_ps, verbose=True)

print('Tomography counts')
for r in range(n_tomo_raw.shape[0]):
    n_tomo_raw[r,:]/=np.sum(n_tomo_raw[r,:])
print(n_tomo_raw)
print('Confusion matrix counts')
for r in range(n_calib.shape[0]):
    n_calib[r,:]/=np.sum(n_calib[r,:])
print(n_calib)
print('Confusion matrix corrected tomo matrix will be')
print(np.around(tomo_analysis.correct_readout_err(n_tomo_raw, n_calib), decimals=5))
# print('Negative counts corrected tomo matrix will be')
# print(tomo_analysis.fix_neg_counts(tomo_analysis.correct_readout_err(n_tomo_raw, n_calib)))

meas_order ['Z' 'X' 'Y']
Tomography counts
[[0.46755 0.53245]
 [0.7505  0.2495 ]
 [0.48565 0.51435]]
Confusion matrix counts
[[0.7619 0.2381]
 [0.2173 0.7827]]
Confusion matrix corrected tomo matrix will be
[[0.45951 0.54049]
 [0.97907 0.02093]
 [0.49275 0.50725]]


In [148]:
# methods = 'Nelder-Mead' 'Powell' 'CG' 'BFGS' 'Newton-CG' 'L-BFGS-B' 'TNC' 'COBYLA' 'SLSQP' 'dogleg' 'trust-ncg'
rho_MLE = tomo_analysis.get_rho_from_counts(
    n_tomo_raw=n_tomo_raw,
    n_calib=n_calib,
    correct_readout=True,
    correct_neg_counts=True,
    method='analytical',
    ZZ_correction=False,
)

if rho_MLE is not None:
    fid_base = qt.fidelity(qt.Qobj(rho_MLE, dims=rho_id.dims), rho_id)**2 # qutip uses N&C fidelity which is "sqrt fidelity"
    purity_base = np.real(np.trace(rho_MLE @ rho_MLE))
    print(f'Fidelity (no ZZ correction): {fid_base}')
    print(f'Purity (no ZZ correction): {purity_base}')
print(np.around(rho_MLE, decimals=3))
savetitle=None
# savetitle=saved_files[-1][:-3]+f'_baseMLE_Q{qubit}.svg'
tomo_analysis.show_plot_rho_2d(rho_MLE, rho_id.full(), title=f'Base MLE (Q{qubit}) Init {init_state}, Play {play_pulses}', cmax=0.5, savetitle=savetitle)

# print('Saved svg:', savetitle)
# print('Saved data:')
# print(*saved_files)

Fidelity (no ZZ correction): 0.9790672053406875
Purity (no ZZ correction): 0.962394613563805
[[0.46 +0.j    0.479+0.007j]
 [0.479-0.007j 0.54 +0.j   ]]


# 2Q tomo

KeyError: 'ZZZ'

In [321]:
tomo_analysis = TomoAnalysis(nb_qubits=2)

In [ ]:
def name_to_state_2q(init_state): # in format |QA>|QB>
    Qa, Qb, _ = init_state.split('>')
    Qa = Qa[1:]
    Qb = Qb[1:]
    psi_name_dict = {
        '0':psiZ[0],
        '1':psiZ[1],
        '0+1':(psiZ[0]+psiZ[1]).unit(),
        '0-1':(psiZ[0]-psiZ[1]).unit(), 
        '0+i':(psiZ[0]+1j*psiZ[1]).unit(), 
        '0-i':(psiZ[0]-1j*psiZ[1]).unit(), 
    }
    try:
        psiA = psi_name_dict[Qa]
    except:
        print(f'QA state {Qa} is not an allowed cardinal state')
    try:
        psiB = psi_name_dict[Qb]
    except:
        print(f'QB state {Qb} is not an allowed cardinal state')
    return qt.tensor(psiA, psiB)

    


In [378]:
psi = name_to_state_2q(init_state)
get_qram_qSLR_state_from_rho(psi)

NameError: name 'id2q' is not defined

In [356]:
tomo_qubits = [0, 1] # which qubit/resonators to do tomo on

init_state = '|0+1>|0+1>' # Q0/Q1 (switch, input)
# init_state = 'Q3Q0_|1>|1>'
# init_state = '|1>|0>' # Q0/Q1 (switch, input)
# init_state = 'Q2Q0_|1>|0>'
# init_state = '|1>|1>' # Q0/Q1 (switch, input)
# play_pulses = [2, 1, 4, 3] # 1: Q2/Q1 with Q0 in g, 2: Q3/Q1 with Q0 in e, 3: Q2 ef pulse, 4: Q3 ef pulse [2, 1, 4, 3]
play_pulses = [0] # 1: Q2/Q1 with Q0 in g, 2: Q3/Q1 with Q0 in e, 3: Q2 ef pulse, 4: Q3 ef pulse [2, 1, 4, 3]
# play_pulses = [2, 4] # 1: Q2/Q1 with Q0 in g, 2: Q3/Q1 with Q0 in e, 3: Q2 ef pulse, 4: Q3 ef pulse [2, 1, 4, 3]
# play_pulses = [1, 3] # 1: Q2/Q1 with Q0 in g, 2: Q3/Q1 with Q0 in e, 3: Q2 ef pulse, 4: Q3 ef pulse [2, 1, 4, 3]
# play_pulses = [2, 4] # 1: Q2/Q1 with Q0 in g, 2: Q3/Q1 with Q0 in e, 3: Q2 ef pulse, 4: Q3 ef pulse [2, 1, 4, 3]
# play_pulses = [2]*8 # 1: Q2/Q1 with Q0 in g, 2: Q3/Q1 with Q0 in e, 3: Q2 ef pulse, 4: Q3 ef pulse [2, 1, 4, 3]

# cool_qubits = [0, 1]
cool_qubits = None


readout_cool = False

use_IQ_pulse = False # state transfer IQ pulses for the state prep
use_robust_pulses = True # use robust pulses for state prep + tomo

post_select = False # this is not the readout post selection

# add_phase = True # phase for swaps
add_phase = False
print('init_state', init_state)
print('post select', post_select)
ps_adjust = [-0.3, -1.0, -0.3, -0.3]
# ps_adjust = [np.inf]*4
ps_qubits = [0, 1, 2, 3]
use_ZZ_mat = True

rho_id = qt.ket2dm(name_to_state_2q(init_state))

evol_mats_path = "S:\\QRAM\\qram_4QR2\\evol_mats"
print('Will save evol mats to path', evol_mats_path)

q0, q1 = tomo_qubits
evol_mats_filename = f'evol_mats_{q0}{q1}.npz'


init_state |0+1>|0+1>
post select False
Will save evol mats to path S:\QRAM\qram_4QR2\evol_mats


In [373]:
qramExpt = meas.QramProtocolExperiment(
        soccfg=soc,
        path=expt_path, 
        config_file=config_path,)


qramExpt.cfg.expt = dict(
        # start=0, # start protocol time [us],
        # step=span/npts, # time step [us], 
        # expts=npts+1, # number of different time experiments - added 1 to include 0 step and full span

        start=np.inf,
        step=0,
        expts=1, # number of different time experiments
        reps=10000, # number of reps per time step,
        # singleshot_reps=100, # reps per state for singleshot calibration
        singleshot_reps=10000, # reps per state for singleshot calibration
        post_process='threshold', # 'threshold', (uses single shot binning), 'scale' (scale by ge_avgs), or None
        tomo_2q=True, # True/False whether to do 2q state tomography on state at last time step
        tomo_qubits=tomo_qubits, # the qubits on which to do the 2q state tomo
        calib_apply_q1_pi2=False, # initialize Q1 to 0+1 for all calibrations
        init_state=init_state,
        play_pulses=play_pulses,
        cool_qubits=cool_qubits,
        
        use_IQ_pulse=use_IQ_pulse, # state transfer IQ pulses for the state prep
        use_robust_pulses=use_robust_pulses, # use robust pulses for state prep + tomo
        plot_IQ=False,
        full_mux_expt = True,
        resonator_reset=[0, 2, 3],
        
        add_phase=add_phase,
        
        # readout_cool=readout_cool,
        # n_init_readout=1,
        # n_trig=1,
        # # init_read_wait_us=5,
        # init_read_wait_us=10,
        
        # Icontrols=I_values_MHz, # 2D array with array of Icontrols for each of IQ_qubits
        # Qcontrols=Q_values_MHz, # 2D array with array of Qcontrols for each of IQ_qubits
        # times_us=times_us,
        # IQ_qubits=IQ_qubits,
        # IQ_gain=[3700, 3950],
        
        # thresholds=[33.169103683345895, 37.49913270024612, 0, 0],
        # angles=[-24.579056838557847, 115.19639444481155, 0, 0],
        # ge_avgs=[[34.43056101813111, -9.525004532775455, 44.589988819153874, -4.651175313807532], [30.053312459336368, -50.17723301886793, 25.548312166558233, -59.65660634352635], array([0., 0., 0., 0.]), array([0., 0., 0., 0.])],
        # counts_calib=[[12553, 5207, 1604, 636], [4673, 12956, 666, 1705], [2241, 876, 12383, 4500], [918, 2095, 4712, 12275]]
 
    )

    # print(qramExpt.cfg)

import Pyro4.util
try:
    qramExpt.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

timesteps [inf]


  0%|          | 0/4 [00:00<?, ?it/s]

Qubit (0)
ge fidelity (%): 76.35894189418944 	 angle (deg): 0.0 	 threshold ge: 769.914512589065
Qubit (1)
ge fidelity (%): 46.78756475647564 	 angle (deg): 0.0 	 threshold ge: 1158.8338955067372
thresholds=[769.914512589065, 1158.8338955067372, 0, 0],
angles=[0.0, 0.0, 0, 0],
ge_avgs=[[0.5557848657445075, -0.907548657445077, -3.530246867371847, -2.7044028478437747], [809.0551288039056, 429.6613794955248, 1098.8818544344997, 710.3305279088689], array([0., 0., 0., 0.]), array([0., 0., 0., 0.])],
counts_calib=[[6670, 2228, 758, 344], [2446, 6427, 312, 815], [948, 326, 6117, 2609], [415, 760, 2400, 6425]]


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

ZZ [2467 2479 2461 2593]
ZX [3751 1349 3063 1837]
ZY [2780 2405 2920 1895]
XZ [4531 4108  648  713]
XX [6430 2263  765  542]
XY [4661 3966  719  654]
YZ [2804 2938 2238 2020]
YX [4213 1538 2759 1490]
YY [3158 2564 2573 1705]


In [374]:
tomo = qramExpt
data = tomo.data
cfg = tomo.cfg
init_state = tomo.cfg.expt.init_state
# qubit = tomo.cfg.expt.qubit
pulse_dict = tomo.pulse_dict
assert np.all(np.array(tomo.meas_order) == tomo_analysis.meas_order)


In [371]:
# run_evol = True 

# if run_evol:
#     evol_mats = tomo_analysis.get_evol_mats_from_yaml(
#         tomo_qubits=tomo_qubits,
#         yaml_cfg=yaml_cfg,
#         pulse_dict=pulse_dict,
#         cutoffs=[4]*(len(tomo_qubits)),
#         soccfg=soc,
#         debug=False,
#         evol_mats_path=evol_mats_path,
#         evol_mats_filename=evol_mats_filename,
#     )
    

    
# else:
#     evol_mats_file_path = os.path.join(evol_mats_path, evol_mats_filename)
#     print(f'Using evol mats from file {evol_mats_file_path}')

#     evol_mats = dict()
#     with np.load(evol_mats_file_path) as npzfile:
#         for key in npzfile.keys():
#             evol_mats.update({key:npzfile[key]})

In [375]:
n_tomo_raw, n_calib = tomo_analysis.n_tomo_calib_from_data(data, cfg, ps_adjust=ps_adjust, ps_qubits=ps_qubits, apply_ps=apply_ps, verbose=True)

# print('Tomography counts')
# for r in range(n_tomo_raw.shape[0]):
#     n_tomo_raw[r,:]/=np.sum(n_tomo_raw[r,:])
# print(n_tomo_raw)
# print('Confusion matrix counts')
# for r in range(n_calib.shape[0]):
#     n_calib[r,:]/=np.sum(n_calib[r,:])
# print(n_calib)
# print('Confusion matrix corrected tomo matrix will be')
# print(np.around(tomo_analysis.correct_readout_err(n_tomo_raw, n_calib), decimals=5))
    
rho_MLE_ZZ = tomo_analysis.get_rho_from_counts(
        n_tomo_raw=n_tomo_raw,
        n_calib=n_calib,
        correct_readout=True,
        correct_neg_counts=True,
        method='analytical',
        ZZ_correction=True,
        evol_mats=evol_mats
    )

print(np.around(rho_MLE_ZZ, decimals=10))
print('list format', np.around(rho_MLE_ZZ, decimals=7).tolist())
fid_ZZ = qt.fidelity(qt.Qobj(rho_MLE_ZZ, dims=rho_id.dims), rho_id)**2 # qutip uses N&C fidelity which is "sqrt fidelity"
purity_ZZ = np.real(np.trace(rho_MLE_ZZ @ rho_MLE_ZZ))
purity_id = np.real(np.trace(rho_id.full() @ rho_id.full()))
print(f'Fidelity (with ZZ correction): {fid_ZZ}')
print(f'Purity (with ZZ correction): {purity_ZZ}')
print(f'Target purity: {purity_id}')

tomo_analysis.show_plot_rho_2d(rho_MLE_ZZ, rho_id.full(), title=f'ZZ MLE (Q{q0}Q{q1}) Init {init_state}, Play {play_pulses}', cmax=0.5, size=(6,6))

 s:\Seb\experiements\qram_tprocv1_expts\TomoAnalysis.py: 1234ComplexWarning: Casting complex values to real discards the imaginary part
 s:\Seb\experiements\qram_tprocv1_expts\TomoAnalysis.py: 1239

[[0.23588204-0.j         0.24086885+0.0101902j  0.22675959-0.01660165j
  0.22029754-0.05502725j]
 [0.24086885-0.0101902j  0.25533584-0.j         0.23175528-0.03220764j
  0.22053161-0.06949515j]
 [0.22675959+0.01660165j 0.23175528+0.03220764j 0.25873293-0.j
  0.20620882-0.06951657j]
 [0.22029754+0.05502725j 0.22053161+0.06949515j 0.20620882+0.06951657j
  0.25004919-0.j        ]]
list format [[(0.235882-0j), (0.2408688+0.0101902j), (0.2267596-0.0166016j), (0.2202975-0.0550273j)], [(0.2408688-0.0101902j), (0.2553358-0j), (0.2317553-0.0322076j), (0.2205316-0.0694951j)], [(0.2267596+0.0166016j), (0.2317553+0.0322076j), (0.2587329-0j), (0.2062088-0.0695166j)], [(0.2202975+0.0550273j), (0.2205316+0.0694951j), (0.2062088+0.0695166j), (0.2500492-0j)]]
Fidelity (with ZZ correction): 0.9232108570553772
Purity (with ZZ correction): 0.8841888783179946
Target purity: 0.9999999999999991


# 3Q tomography

In [382]:
tomo_analysis = TomoAnalysis(nb_qubits=3)

# 2 cardinal points on Bloch sphere for each measurement axis
# psiZ = [qt.basis(3,0), qt.basis(3,1)]
psiZ = [qt.basis(2,0), qt.basis(2,1)]
psiX = [1/np.sqrt(2)*(psiZ[0]+psiZ[1]), 1/np.sqrt(2)*(psiZ[0]-psiZ[1])]
psiY = [1/np.sqrt(2)*(psiZ[0]+1j*psiZ[1]), 1/np.sqrt(2)*(psiZ[0]-1j*psiZ[1])]
psi_dict = dict(Z=psiZ, X=psiX, Y=psiY)

psis = dict() # psis in the ZZZ basis
for i, label_numeric in enumerate(tomo_analysis.calib_order_numeric):
    psis.update({label_numeric:tomo_analysis.psi_basis['ZZZ'][i]})
id3q = qt.tensor(qt.qeye(2), qt.qeye(2), qt.qeye(2))
id2q = qt.tensor(qt.qeye(2), qt.qeye(2))

# psi_basis_3q = dict() # 27*8 elements
# # psi_tensor = [psiZ[0], psiZ[0], psiZ[0], psiZ[0]]
# S123 = 'ZZZ'
# S1, S2, S3 = S123
# psi1 = psi_dict[S1]
# psi2 = psi_dict[S2]
# psi3 = psi_dict[S3]

# psi_basis_3q.update({S123:[]})

# for state1 in (0,1):
#     for state2 in (0,1):
#         for state3 in (0,1):
#             psi_basis_3q[S123].append(qt.tensor(psi1[state1], psi2[state2], psi3[state3]).unit())
# calib_order_numeric_3q = ['000', '001', '010', '011', '100', '101', '110', '111']
# psis = dict() # psis in the ZZZ basis
# for i, label_numeric in enumerate(calib_order_numeric_3q):
#     psis.update({label_numeric:psi_basis_3q['ZZZ'][i]})
# id3q = qt.tensor(qt.qeye(2), qt.qeye(2), qt.qeye(2))

def get_qram_qSLR_state_from_rho(init_rho_SI, post_select=False, post_select_state=None):
    """
    init_rho_SI should be a np array
    """
    # in order |00>, |01>, |10>, |11> for Q0 (switch), Q1 (input)
    SI_to_SLR = [psis['000'], -1j*psis['010'], psis['100'], -1j*psis['101']]

    assert np.shape(init_rho_SI) == id2q.shape
    rho_final_SLR = 0*qt.ket2dm(psis['000'])
    print('constructing final state on |switch, out1, out2>')
    for i in range(len(SI_to_SLR)):
        slr_ket_i = SI_to_SLR[i]
        for j in range(len(SI_to_SLR)):
            slr_ket_j = SI_to_SLR[j]
            mat_el = init_rho_SI[i, j]
            rho_final_SLR += mat_el * slr_ket_i * slr_ket_j.dag()
    return rho_final_SLR.unit()

In [389]:
psi = name_to_state_2q(init_state)
rho = get_qram_qSLR_state_from_rho(psi*psi.dag())

constructing final state on |switch, out1, out2>


In [261]:
# init_states = ['|0+1>|0>', '|0>|0+1>', '|1>|0+1>', '|0+1>|1>', '|0+1>|0+1>', '|0>|0>', '|1>|0>', '|1>|1>', '|0>|1>']
init_state = '|1>|1>'
# init_states = ['|0+1>|0>', '|0>|0+1>', '|1>|0+1>', '|0+1>|1>', '|0+1>|0+1>']
# init_states = ['|0+1>|0>', '|0>|0+1>', '|1>|0+1>', '|0+1>|1>']
# init_states = ['|0+1>|0+1>']
# protocol_play =  [1, 3] # 1: Q2/Q1 with Q0 in g, 2: Q3/Q1 with Q0 in e, 3: Q2 ef pulse, 4: Q3 ef pulse [2, 1, 4, 3]
protocol_play = [2, 1, 4, 3] # 1: Q2/Q1 with Q0 in g, 2: Q3/Q1 with Q0 in e, 3: Q2 ef pulse, 4: Q3 ef pulse [2, 1, 4, 3]

sync_between_swaps = False
sync_between_efs = False

tomo_qubits = [0, 2, 3] # which qubit/resonators to do tomo on
save_data = False
saved_files = []
# cool_qubits = [0, 1]
cool_qubits = None

readout_cool = False

use_IQ_pulse = False # state transfer IQ pulses for the state prep
use_robust_pulses = True # use robust pulses for state prep + tomo

print('init_states', init_state)
print('protocol_play', protocol_play)
print('sync between swaps', sync_between_swaps)
print('sync between efs', sync_between_efs)
print('readout cool', readout_cool)
print('use robust pulses', use_robust_pulses)
print('save data', save_data)
if not save_data:
    print('WARNING!! YOU ARE NOT SAVING DATA!')

init_states |1>|1>
protocol_play [2, 1, 4, 3]
sync between swaps False
sync between efs False
readout cool False
use robust pulses True
save data False
WARNING!! YOU ARE NOT SAVING DATA!


In [262]:
qramExpt = meas.QramProtocol3QTomoExperiment(
        soccfg=soc,
        path=expt_path,
        config_file=config_path,
    )
    
qramExpt.cfg.expt = dict(
        start=np.inf,
        step=0,
        expts=1, # number of different time experiments
    
        reps=5000, # number of reps per tomo/time step,
        singleshot_reps=5000, # reps per state for singleshot calibration
        # reps=100, # number of reps per tomo/time step,
        # singleshot_reps=100, #000, # reps per state for singleshot calibration
        post_process=None, # 'threshold', (uses single shot binning), 'scale' (scale by ge_avgs), or None
        tomo_3q=True, # flag to perform tomo on last time step
        tomo_qubits=tomo_qubits, # the qubits on which to do the 3q state tomo
        init_state=init_state,
        play_pulses=protocol_play, # [2, 1, 4, 3]
        cool_qubits=cool_qubits,
        sync_between_swaps=sync_between_swaps,
        sync_between_efs=sync_between_efs,
        
        use_IQ_pulse=use_IQ_pulse, # state transfer IQ pulses for the state prep
        use_robust_pulses=use_robust_pulses, # use robust pulses for state prep + tomo
        plot_IQ=False,
        
        readout_cool=readout_cool,
        n_init_readout=1,
        n_trig=1,
        init_read_wait_us=5,
        
        full_mux_expt = True,
        resonator_reset=[0, 2, 3],
    )
    

In [263]:
try:
    qramExpt.acquire(progress=True, debug=False)
    success = True
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))
    
if save_data: saved_files.append(qramExpt.save_data())

timesteps [inf]


  0%|          | 0/8 [00:00<?, ?it/s]

Qubit  (0)
ge fidelity (%): 70.26 	 angle (deg): 0.0 	 threshold ge: 566.5987055395138
Qubit  (2)
ge fidelity (%): 79.41786357271452 	 angle (deg): 0.0 	 threshold ge: 910.0655573765281
Qubit  (3)
ge fidelity (%): 73.53999999999998 	 angle (deg): 0.0 	 threshold ge: 794.3194434509765
thresholds=[566.5987055395138, 0, 910.0655573765281, 794.3194434509765],
angles=[0.0, 0, 0.0, 0.0],
ge_avgs=[[-1.2812465419039873, -0.6262017900732303, 0.4882675345809606, -4.987295687550855], [0, 0, 0, 0], [-7.669710333604558, -2.1683025223759156, -1.9803583401139135, 3.9557332790886903], [-4.39679739625712, -5.701430105777056, 2.31208413344182, -11.482829292107402]] ,
counts_calib=[[3263, 474, 414, 67, 626, 72, 69, 15], [531, 3264, 56, 375, 101, 595, 12, 66], [366, 43, 3351, 495, 49, 14, 585, 97], [67, 380, 580, 3195, 11, 53, 102, 612], [518, 92, 85, 15, 3286, 488, 435, 81], [123, 470, 21, 76, 750, 3074, 95, 391], [72, 14, 547, 101, 371, 55, 3316, 524], [18, 85, 83, 570, 69, 402, 528, 3245]]



  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

In [264]:
# run_evol = True 

# if run_evol:
#     evol_mats = tomo_analysis.get_evol_mats_from_yaml(
#         tomo_qubits=tomo_qubits,
#         yaml_cfg=yaml_cfg,
#         pulse_dict=pulse_dict,
#         cutoffs=[4]*(len(tomo_qubits)),
#         soccfg=soc,
#         debug=False,
#         evol_mats_path=evol_mats_path,
#         evol_mats_filename=evol_mats_filename,
#     )
    

In [265]:
data = qramExpt.data
cfg = qramExpt.cfg
ps_adjust = [0, 0, 0, 0]
ps_qubits = [0, 1, 2, 3]
apply_ps = False
n_tomo_raw, n_calib = tomo_analysis.n_tomo_calib_from_data(data, cfg, ps_adjust=ps_adjust, ps_qubits=ps_qubits, apply_ps=apply_ps, verbose=False)

print('Tomography counts')
for r in range(n_tomo_raw.shape[0]):
    print(tomo_analysis.meas_order[r])
    print(n_tomo_raw[r,:]/np.sum(n_tomo_raw[r,:]))
    
print('Confusion matrix counts')
for r in range(n_calib.shape[0]):
    print(tomo_analysis.calib_order[r])
    print(n_calib[r,:]/np.sum(n_calib[r,:]))
 
 
 

# ------------------------------ #
# Base MLE
# ------------------------------ #

rho_MLE_base = tomo_analysis.get_rho_from_counts(
    n_tomo_raw=n_tomo_raw,
    n_calib=n_calib,
    correct_readout=True,
    correct_neg_counts=True,
    method='analytical',
    ZZ_correction=False,
    verbose=False,
)

# fid_base = qt.fidelity(qt.Qobj(rho_MLE_base, dims=rho_id.dims), rho_id)**2 # qutip uses N&C fidelity which is "sqrt fidelity"
# purity_base = np.real(np.trace(rho_MLE_base @ rho_MLE_base))
# purity_id = np.real(np.trace(rho_id.full() @ rho_id.full()))

Tomography counts
ZZZ
[0.024  0.0828 0.0332 0.0158 0.114  0.5776 0.0506 0.102 ]
ZZX
[0.0396 0.0726 0.0266 0.0244 0.231  0.4544 0.056  0.0954]
ZZY
[0.0322 0.0662 0.026  0.0254 0.2292 0.4568 0.0556 0.1086]
ZXZ
[0.0282 0.049  0.034  0.0506 0.0728 0.3236 0.0916 0.3502]
ZXX
[0.0246 0.0542 0.032  0.057  0.111  0.2758 0.163  0.2824]
ZXY
[0.0284 0.0444 0.0366 0.0514 0.1426 0.2448 0.1378 0.314 ]
ZYZ
[0.0134 0.0486 0.037  0.049  0.0728 0.3166 0.0948 0.3678]
ZYX
[0.0198 0.04   0.0316 0.0544 0.125  0.28   0.1598 0.2894]
ZYY
[0.0134 0.0398 0.0432 0.0518 0.1202 0.2712 0.1742 0.2862]
XZZ
[0.047  0.3064 0.0412 0.0516 0.0824 0.353  0.0538 0.0646]
XZX
[0.1132 0.2498 0.0322 0.056  0.1524 0.2972 0.0372 0.062 ]
XZY
[0.1138 0.2512 0.0322 0.0504 0.1582 0.2876 0.0434 0.0632]
XXZ
[0.0342 0.1794 0.0588 0.192  0.0552 0.1904 0.079  0.211 ]
XXX
[0.0614 0.1516 0.0776 0.1504 0.083  0.182  0.109  0.185 ]
XXY
[0.0724 0.1422 0.076  0.1762 0.089  0.16   0.088  0.1962]
XYZ
[0.0362 0.1832 0.0522 0.1948 0.0502 0.1958 0.070

In [266]:
tomo_analysis.show_plot_rho_2d(rho_MLE_base, None, title=f'Init {init_state}, Play {protocol_play}', cmax=0.5, size=(7,5))

## Several input state

In [ ]:
init_states = ['|0>|0+1>', '|1>|0+1>', '|0+1>|1>', '|0+1>|0+1>']

